In [1]:
from dp import is_running_on_ipython, KoMRC, Preprocessor, Postprocessor
if is_running_on_ipython():
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

from transformers import (
    ElectraForQuestionAnswering,
    ElectraTokenizerFast
)
import torch

import numpy as np
import pandas as pd
import csv
import random

# Google drive mount
# drive.mount('/content/drive')
import os
from os.path import join
# change for your path
# ckpt_dir = join('/', 'content', 'drive', 'My Drive', 'goorm K-Digital', '자연어처리', 'Project2')
ckpt_dir = 'ckpt'

# seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Load a test dataset

In [2]:
data_dir = 'data'

tokenizer = ElectraTokenizerFast.from_pretrained('monologg/koelectra-base-v3-finetuned-korquad')
preprocessor = Preprocessor(tokenizer, 512, 128, 'right')
postprocessor = Postprocessor(tokenizer)
test_dataset = KoMRC.load(join(data_dir, 'test.json'))

# Load a pretrained model

In [3]:
model = ElectraForQuestionAnswering.from_pretrained(join(ckpt_dir, 'LR5e-05_WD0.1_WR0.5_DS128.bin'))
model.cuda()
model.eval()

ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [4]:
output_path = join(data_dir, 'output', 'result.csv')
os.makedirs(join(data_dir, 'output'), exist_ok=True)

verbose = False
with torch.no_grad(), open(output_path, 'w', encoding = 'utf-8-sig') as fd:
    writer = csv.writer(fd)
    writer.writerow(['Id', 'Predicted'])

    rows = []
    for data in tqdm(test_dataset, "Testing"):
        if verbose: print(f'질문: {data["question"]}')
        tokenized_example = tokenizer(data['question'], 
                                      data['context'], 
                                      truncation="only_second",
                                      max_length=512,
                                      stride=128, 
                                      return_overflowing_tokens=True,
                                      return_offsets_mapping=True,
                                      padding='max_length'
                                      )
        input_ids, token_type_ids = [
            torch.tensor(tokenized_example[key], dtype=torch.long, device="cuda")
            for key in ("input_ids", "token_type_ids")
        ]
        offset_mapping = tokenized_example['offset_mapping']

        start_logits, end_logits = model(input_ids=input_ids, token_type_ids=token_type_ids, return_dict=False)
        start_logits = start_logits.cpu()
        end_logits = end_logits.cpu()
        
        answer = postprocessor.eval(input_ids, start_logits, end_logits, max_len=40, verbose=verbose)
        if verbose: print(answer, '\n')
        
        rows.append([data['guid'], answer])
    
    writer.writerows(rows)

Testing:   0%|          | 0/4008 [00:00<?, ?it/s]

질문: 말라카이트에서 나온 색깔을 사용한 에디션은?
raw answers: ['뉴 740Li 25주년 에디션']
뉴 740Li 25주년 에디션

질문: 사고 비행기의 목적지는?
raw answers: ['독일 뒤셀도르프로']
독일 뒤셀도르프로

질문: 2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?
raw answers: ['페이스북', '']
페이스북

질문: 포브스의 2014년 일하고 싶은 50대 회사 조사에서 5위를 한 기업은?
raw answers: ['페이스북', '']
페이스북

질문: 호세 카레라스가 재단의 도움을 받아 병을 치료한 병원의 소재지는?
raw answers: ['마드리드']
마드리드

질문: 러시아 펀드 수익률 하락의 원인은?
raw answers: ['국제 원자재값 하락', '국제 원자재값 하락']
6.090220928192139 10.823078155517578 국제 원자재값 하락

질문: 코레일이 증자해서 만들고자 하는 자본금 액수는?
raw answers: ['2조6000억원', '']
2조6000억원

질문: 필리핀 숙박업소가 손님을 다시 받으려면 필수적으로 떼야 하는 서류는?
raw answers: ['‘ 운영허가 인증서 ’', '']
‘ 운영허가 인증서 ’

질문: 실리콘을 실리콘유로 만들기 위해 거치는 과정은?
raw answers: ['그물 모양 구조']
그물 모양 구조

질문: 칼 구스타브 에밀 만네르헤임 남작이 지원하는 자원 봉사단은 어디로 보내졌나?
raw answers: ['노르웨이']
노르웨이

질문: 착한 포장 2차 프로젝트에 우선 적용될 제품 수는?
raw answers: ['', '22개']
22개

질문: 김윤이 소속된 보이그룹이 kpop 아시아 투어를 진행한 첫 번째 나라는?
raw answers: ['필리핀']
필리핀

질문: 1930년대 초중반에 전수린과 함께 작곡가로 쌍두마차를 이룬 사람은?
raw answers: ['김교성', '']
김교성

질문: 16일에 슈에무라 제품이 

KeyboardInterrupt: 